In [1]:
import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections
import os

from collections import OrderedDict
import uproot
import pandas as pd

import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from helper_functions import deltaR, deltaPhi

import CMS_lumi, tdrstyle
a = tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0

wH = 1
Z_MASS = 91.2


# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.18/02
3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [2]:
fpath =OrderedDict()
tree = OrderedDict()


start_t = time.time()

isData = 1
if isData:
    ntupler_version = 'V1p17/'

    analyzer_version = "/v5/v162/"

    data_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/Data2018/'+analyzer_version+'/normalized/'

    fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root'


else:

    old_ctau = {
#     'STodd_ms3p0':[100],
    'STodd_ms7':[1000],
    'STodd_ms15':[1000],
    'STodd_ms40':[1000],
    'STodd_ms55':[1000],
    'SToBB_ms15':[1000],
    'SToBB_ms40':[1000],
    'SToBB_ms55':[1000],
    'SToTauTau_ms7':[1000],
    'SToTauTau_ms15':[1000],
    'SToTauTau_ms40':[1000],
    'SToTauTau_ms55':[1000],
    'SToEE_ms0p4':[10],
    'SToGammaGamma_ms0p4':[10],
    'SToKPlusKMinus_ms1p5':[187],
    'SToK0K0_ms1p5':[187],
    'SToPi0Pi0_ms1p0':[125],
    'SToPiPlusPiMinus_ms1p0':[125],
        
        
    }

#     for mass in sample.keys():
#         for ct in sample[mass]:

#             if year_index == 3:
#                 mc_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
#                 fpath_bkg['m'+mass+'ctau'+ct] = mc_path + 'ggH_HToSSTo'+decay+'_MH-125_MS-'+str(mass)+'_ctau-'+ct+'_137000pb_weighted.root'
#             else:
#                 mc_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/'+years[year_index]+'/'+analyzer_version+'/normalized/'
#                 fpath_bkg['m'+mass+'ctau'+ct] = mc_path + 'ggH_HToSSTo'+decay+'_MH-125_MS-'+str(mass)+'_ctau-'+ct+'_'+tune[year_index]+'_'+str(lumi[year_index])+'pb_weighted.root'



    ntupler_version = 'V1p17/'
    mc_path = {}
    analyzer_version = 'v1/v169/'
    mc_central_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
    analyzer_version = 'v2/v169/'
    mc_private_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'


    for k in old_ctau.keys():
        for ct in old_ctau[k]:

            key = 'MC_ggH_'+k + '_'+str(ct)
            if 'ms7' in k or 'ms15' in k or 'ms40' in k or 'ms55' in k:
                mass = k[k.find('ms')+2:]
                if 'dd' in k:fpath[key] = mc_central_path+'ggH_HToSSTodddd_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
                elif 'BB' in k:fpath[key] = mc_central_path+'ggH_HToSSTobbbb_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
                else:fpath[key] = mc_central_path+'ggH_HToSSTo4Tau_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
            else:
                fpath[key] = mc_private_path+'ggH_HToSS_'+k+'_pl'+str(ct)+'_137000pb_weighted.root'
            if not os.path.exists(fpath[key]):print(key, fpath[key])


            
                
                
NEvents = {}
Total = {}
accep = {}
accep_met = {}
for k,v in fpath.items():
    print (k, v)
    root_dir = uproot.open(v) 
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    if not isData:
#         Total[k] = root_dir['Total'][1]
#         Total[k] = root_dir['Total'][1]


        accep[k] = root_dir['acceptance'][1]
        accep_met[k] = root_dir['acceptance_met'][1]
#     else:
#         Nmet200 = root_dir['Nmet200'][1]
#         NmetFilter = root_dir['NmetFilter'][1]
#         Nlep0 = root_dir['Nlep0'][1]
#         Njet1 = root_dir['Njet1'][1]
#         NcosmicVeto = root_dir['NcosmicVeto'][1]
    a = tree[k]["weight"].array()
    print("NEvents",NEvents[k],len(a))


data /storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17//Data2018//v5/v162//normalized/Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root
NEvents 22763250.0 14129474


# background cut flow

In [3]:
JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False
cut_based = True
cut_based_version="v4"

intime = True
DPHI_CUT = 1
cscRechitClusterNStation10 = {}
cscRechitClusterAvgStation10 = {}
cscRechitClusterEta = {}
sel_ev = {}
weight = {}
dphiMet_cluster = {}
bdt_sel = {}
Nrechits = {}
ncluster = 1
cluster_index = ''
table = OrderedDict()

table['met'] = "Trigger and MET "
table['met_filter'] = "MET filters "
table['jet'] = r"$N_{jet} \geq 1$ "
table['cosmic_veto'] = r"$N_{\text{CSC+DT rings}} \leq 10 $ "
table['spike'] = "DT noise veto"
table['2_cluster'] = "$N_{cluster} \geq 2$ "
table['dt_cosmic_veto'] = "DT cosmic veto"
table['muon_veto'] = "muon veto "
table['jet_veto'] = "jet veto "
table['me11_ratio'] = "ME11/MB1 ratio "
table['station_ratio'] = "N_{min station hits}/N_{max station hits} "
table['time'] = "time cut "
table['time_spread'] = "time spread "
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['deltaR'] = r"$\Delta R \mathrm{ (cluster1,cluster2)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "

k = 'data'
T = tree[k] 
for category in [ 0,1,2]:

    
    if category == 0:
        NRECHIT1 = 100
        NRECHIT2 = 100
    elif category == 1:
        NRECHIT1 = 80
        NRECHIT2 = 80
    else:
        NRECHIT1 = 80
        NRECHIT2 = 100
    weight = T.array('weight')*0.0+1.0
    print(weight[:10], len(weight))
    ########### SELECTION: JETS ############

    sel_jet = np.logical_and(T.array('jetPt') >= 30, np.abs(T.array('jetEta')) < 2.4 )
    sel_jet = np.logical_and(sel_jet, T.array('jetTightPassId'))

    ########### SELECTION: EVENTS ############
    NEvents[k] = len(weight)
    sel_ev[k] = np.logical_and(T.array('metEENoise')>=200 ,T.array('METNoMuTrigger'))
    print(len(sel_ev[k]),np.count_nonzero(sel_ev[k]))
    print("trigger + MET", np.count_nonzero(sel_ev[k])/NEvents[k], np.count_nonzero(sel_ev[k])/NEvents[k])
#     table["met"] += " & {0:6.3f} & {1:6.3f} & {2:6.0f}".format(100* np.count_nonzero(sel_ev[k])/NEvents[k], 100* np.count_nonzero(sel_ev[k])/NEvents[k], np.count_nonzero(sel_ev[k]))
    table["met"] += " & {0:6.2f} & {1:6.0f}".format(100* np.count_nonzero(sel_ev[k])/NEvents[k], np.count_nonzero(sel_ev[k]))



    acc_met = np.count_nonzero(sel_ev[k])
    sel_temp = np.count_nonzero(sel_ev[k])


    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('Flag2_all') == 1)
    print("MET filters", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
#     table["met_filter"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(sel_ev[k])/sel_temp, 100*np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k]))
    table["met_filter"] += " & {0:6.2f} & {1:6.0f}".format(100* np.count_nonzero(sel_ev[k])/sel_temp, np.count_nonzero(sel_ev[k]))
    sel_temp = np.count_nonzero(sel_ev[k])


    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    print("1 jet", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
#     table["jet"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(sel_ev[k])/sel_temp, 100*np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k]))
    table["jet"] += " & {0:6.2f} & {1:6.0f}".format(100* np.count_nonzero(sel_ev[k])/sel_temp, np.count_nonzero(sel_ev[k]))

    sel_temp = np.count_nonzero(sel_ev[k])


    sel_ev[k] = np.logical_and(sel_ev[k] , T.array("nDtRings")+T.array("nCscRings") < 10)
    print("cosmic shower veto", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
#     table["cosmic_veto"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(sel_ev[k])/sel_temp, 100*np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k]))
    table["cosmic_veto"] += " & {0:6.2f} & {1:6.0f}".format(100* np.count_nonzero(sel_ev[k])/sel_temp, np.count_nonzero(sel_ev[k]))

    sel_temp = np.count_nonzero(sel_ev[k])

    
    spike = np.logical_and( T.array('nDTRechitsSector')[:,0,0,7]>50,  T.array('nDTRechitsSector')[:,0,0,7]+T.array('nDTRechitsSector')[:,0,0,8]+T.array('nDTRechitsSector')[:,0,0,9]>120)
    spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,8]>25)
    spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,9]>10)
    sel_ev[k] = np.logical_and(sel_ev[k] , np.logical_not(spike))
    print("spike", np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k])/sel_temp,np.count_nonzero(sel_ev[k]))
#     table["spike"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.count_nonzero(sel_ev[k])/sel_temp, 100*np.count_nonzero(sel_ev[k])/acc_met, np.count_nonzero(sel_ev[k]))
    table["spike"] += " & {0:6.2f} & {1:6.0f}".format(100* np.count_nonzero(sel_ev[k])/sel_temp, np.count_nonzero(sel_ev[k]))

    sel_temp = np.count_nonzero(sel_ev[k])

    ########### SELECTION: CLUSTERS ############

    # blind bin D
    if category == 0: 
        sel_ev[k] = np.logical_and(sel_ev[k], (T.array('cscRechitCluster' + cluster_index + 'Size')>=100).sum()<2)
    elif category == 1: 
        sel_ev[k] = np.logical_and(sel_ev[k], (T.array('dtRechitCluster' + cluster_index + 'Size')>=80).sum()<2)
    else: 
        sel_ev[k] = np.logical_and(sel_ev[k], np.logical_not(np.logical_and((T.array('cscRechitCluster' + cluster_index + 'Size')>=100).sum()>=1,\
                                                             (T.array('dtRechitCluster' + cluster_index + 'Size')>=80).sum()>=1)))
    if category == 0:
        sel_rechitcluster = T.array('cscRechitCluster' + cluster_index + 'Size')>=50
#         print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)

        table["2_cluster"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


        sel_rechitcluster = np.logical_and( sel_rechitcluster, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 30, T.array('cscRechitClusterMuonVetoGlobal'))))
        print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["muon_veto"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))          
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

        
        sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') < 30)
        print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["jet_veto"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))          
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


        sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'Me11Ratio') < 1)
        print("me11 ratio", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["me11_ratio"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))          
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
        
        sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') < 12.5, T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') > -5.0))
        print("time cut", np.sum([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["time"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))          
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


        sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'TimeSpreadWeightedAll') < 20)
        print("time spread cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["time_spread"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))          
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


        
        sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1.2)
        print("dphi cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["dphi"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))          
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
  
        temp = np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)
        cond = deltaR(T.array('cscRechitClusterEta')[sel_rechitcluster][temp][:,0], T.array('cscRechitClusterPhi')[sel_rechitcluster][temp][:,0],
                                                     T.array('cscRechitClusterEta')[sel_rechitcluster][temp][:,1], T.array('cscRechitClusterPhi')[sel_rechitcluster][temp][:,1])<2
        sel_ev[k][temp] = cond
        print("deltaR cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)

        table["deltaR"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))          
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

                           
        sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'Size')) >= 100)
        print("nrechits cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["nrechits"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))          
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

        table['station_ratio'] += "&  /  & /"
        table['dt_cosmic_veto'] += "&  /  & /"

    if category == 1:
        sel_rechitcluster = T.array('dtRechitCluster' + cluster_index + 'Size')>=0
        print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)

        table["2_cluster"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

        
        sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
        sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
        sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
        sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
                                   T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
        nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
        +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)

        sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))    
        print("dt cosmic veto cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["dt_cosmic_veto"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
        
        
        
    
        sel_rechitcluster = np.logical_and( sel_rechitcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))
        print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["muon_veto"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

        
        sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitCluster' + cluster_index + 'JetVetoPt') < 50)
        print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["jet_veto"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

        sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))
        print("MB1 ratio", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["me11_ratio"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
        
        cut = 5
        station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
    +(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)
        max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
        min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
        sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_or(station<4, min_station/max_station<0.4)) #remove if both clusters are 4 stations
        print("min/max station ratio", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["station_ratio"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
        
        
        sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('dtRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1)
        print("dphi cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["dphi"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
  
        sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('dtRechitCluster' + cluster_index + 'Size')) >= 80)
        print("nrechits cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
        table["nrechits"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()==2)])
        print(np.count_nonzero(sel_temp))

        table['time'] += " & / & / "
        table['time_spread'] += " & / & /"
        table['deltaR'] += "& / & /"
    if category == 2:
        sel_csc = T.array('cscRechitCluster' + cluster_index + 'Size')>=50
        sel_dt = T.array('dtRechitCluster' + cluster_index + 'Size')>=50
#         sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(binD))
        print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

        table["2_cluster"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])

        
                
        sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
        sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
        sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
        sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
                                   T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
        nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
        +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)

        sel_dt = np.logical_and(sel_dt, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))     
        print("dt cosmic veto", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
        table["dt_cosmic_veto"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])
        
        
        sel_dt = np.logical_and( sel_dt, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))
        sel_csc = np.logical_and( sel_csc, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 30, T.array('cscRechitClusterMuonVetoGlobal'))))
        print("muon veto", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
        table["muon_veto"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])

        
        sel_dt = np.logical_and( sel_dt, T.array('dtRechitCluster' + cluster_index + 'JetVetoPt') < 50)
        sel_csc = np.logical_and(sel_csc, T.array('cscRechitClusterJetVetoPt') < 30)
        print("jet veto", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
        table["jet_veto"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])
        
        
        
        sel_csc = np.logical_and( sel_csc, T.array('cscRechitClusterMe11Ratio') < 1)
        sel_dt = np.logical_and(sel_dt, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))
        print("mb1 ratio", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
        table["me11_ratio"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])
        
        
        cut = 5
        station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
    +(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)
        max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
        min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
        sel_dt = np.logical_and(sel_dt, np.logical_or(station<4, min_station/max_station<0.4)) #remove if both clusters are 4 stations
        print("station cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
        table["station_ratio"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])

        
        
        sel_csc = np.logical_and(sel_csc, np.logical_and(T.array('cscRechitClusterTimeWeighted') < 12.5, T.array('cscRechitClusterTimeWeighted') > -5.0))
        print("time cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
        table["time"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])

        
        sel_csc = np.logical_and(sel_csc, T.array('cscRechitClusterTimeSpreadWeightedAll')<20)
        print("time spread", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
        table["time_spread"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])
        
        
        sel_dt = np.logical_and( sel_dt, np.abs(T.array('dtRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1)
        sel_csc = np.logical_and( sel_csc, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1.2)
        print("dphi cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

        table["dphi"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


        temp = np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))
        cond = deltaR(T.array('dtRechitClusterEta')[sel_dt][temp][:,0], T.array('dtRechitClusterPhi')[sel_dt][temp][:,0],
                                                     T.array('cscRechitClusterEta')[sel_csc][temp][:,0], T.array('cscRechitClusterPhi')[sel_csc][temp][:,0])<2.5
  
        sel_ev[k][temp] = cond

        print("deltaR cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

        table["deltaR"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])
  
        sel_dt = np.logical_and( sel_dt, np.abs(T.array('dtRechitCluster' + cluster_index + 'Size')) >= 80)
        sel_csc = np.logical_and( sel_csc, np.abs(T.array('cscRechitCluster' + cluster_index + 'Size')) >= 100)
        print("nrechits", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

        table["nrechits"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                         np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))]))  
        sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()==1, sel_dt.sum()==1))])
        print(np.count_nonzero(sel_temp))


for k,v in table.items():
    print(v+r" \\")



[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] 14129474
14129474 14129474
trigger + MET 1.0 1.0
MET filters 0.9261875565926941 0.9261875565926941 13086543
1 jet 0.8087214711602144 0.8731724642634804 11426809
cosmic shower veto 0.8087169416214645 0.9999943991362769 11426745
spike 0.8086853763983005 0.9999609687623203 11426299
muon cut 0.0018389927324966237 0.8794422
jet cut 0.00041792072372970145 0.22725524
me11 ratio 0.00023801310650346926 0.5695174
time cut 2.7035684414012865e-05 0.113589056
time spread cut 9.200625585920608e-06 0.34031415
dphi cut 1.6278029882782615e-06 0.17692308
deltaR cut 4.954183007803405e-07 0.3043478
nrechits cut 0.0 0.0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] 14129474
14129474 14129474
trigger + MET 1.0 1.0
MET filters 0.9261875565926941 0.9261875565926941 13086543
1 jet 0.8087214711602144 0.8731724642634804 11426809
cosmic shower veto 0.8087169416214645 0.9999943991362769 11426745
spike 0.8086853763983005 0.9999609687623203 11426299
cluster efficiency 0.001358507754782662 0.0016798

# signal

In [65]:

for sample in old_ctau.keys(): 
    
    mass = sample[sample.find('ms')+2:].replace('p','.')
    if float(mass) == int(float(mass)): mass = str(int(float(mass)))
#     print(sample,mass)
    for ct_int in old_ctau[sample]: 
     
        ct = str(ct_int)
        k = 'MC_ggH_'+sample+'_'+ct
        T = tree[k]

        sel_ev = {}
        weight = {}
        dphiMet_cluster = {}
        bdt_sel = {}
        Nrechits = {}
        ncluster = 1
        cluster_index = ''
        table = OrderedDict()
        table['acc'] = 'acceptance'
        table['met'] = "Trigger and MET "
        table['met_filter'] = "MET filters "
        table['jet'] = r"$N_{jet} \geq 1$ "
        table['cosmic_veto'] = r"$N_{\text{CSC+DT rings}} \leq 10 $ "
        table['spike'] = "DT noise veto"
        table['2_cluster'] = "$N_{cluster} \geq 2$ "
        table['dt_cosmic_veto'] = "DT cosmic veto"
        table['muon_veto'] = "muon veto "
        table['jet_veto'] = "jet veto "
        table['me11_ratio'] = "ME11/MB1 ratio "
        table['station_ratio'] = r"$N_{\mathrm{min station hits}}$/$N_{\mathrm{max station hits}}$ "
        table['time'] = "time cut "
        table['time_spread'] = "time spread "
        table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
        table['deltaR'] = r"$\Delta R \mathrm{ (cluster1,cluster2)}$ "
        table['nrechits'] = r"$N_{rechits}$ cut "

        

        for category in [0, 1, 2]:
#             if not category == 1:continue
            if category == 0:
                NRECHIT1 = 100
                NRECHIT2 = 100
            elif category == 1:
                NRECHIT1 = 80
                NRECHIT2 = 80
            else:
                NRECHIT1 = 80
                NRECHIT2 = 100
            weight = T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF')
            
            ########### SELECTION: JETS ############

            sel_jet = np.logical_and(T.array('jetPt') >= 30, np.abs(T.array('jetEta')) < 2.4 )
            sel_jet = np.logical_and(sel_jet, T.array('jetTightPassId'))

            ########### SELECTION: EVENTS ############
            if category == 0: 
                sel_ev[k] = np.sum(T.array('gLLP_csc'),axis = 1) == 2
            elif category == 1:
#                 sel_ev[k] = np.sum(T.array('gLLP_dt'),axis = 1) == 2 
                sel_llp = np.logical_and(T.array('gLLP_decay_vertex_r')<800, T.array('gLLP_decay_vertex_r')>200)
                sel_llp = np.logical_and(sel_llp, np.abs(T.array('gLLP_decay_vertex_z'))<661)
#                 sel_llp = np.logical_and(sel_llp, np.logical_not(T.array('gLLP_csc')))
                sel_ev[k] = np.sum(sel_llp, axis = 1) == 2
                
            else: 
                sel_llp = np.logical_and(T.array('gLLP_decay_vertex_r')<800, T.array('gLLP_decay_vertex_r')>200)
                sel_llp = np.logical_and(sel_llp, np.abs(T.array('gLLP_decay_vertex_z'))<661)
#                 sel_llp = np.logical_and(sel_llp, np.logical_not(T.array('gLLP_csc')))
                
                a = np.logical_and(T.array('gLLP_csc')[:,0] == 1, sel_llp[:,1] == 1)
                b = np.logical_and(T.array('gLLP_csc')[:,1] == 1, sel_llp[:,0] == 1)
                sel_ev[k] = np.logical_or(a, b)
#                 sel_ev[k] = np.sum(sel_llp, axis = 1) >= 0

            acceptance = np.sum((T.array('weight')*T.array('higgsPtWeight'))[sel_ev[k]])
#             total = np.sum(T.array('weight'))
            total = 48.58*137000
#             print(np.sum(T.array('weight')), total)
            table["acc"] += " & {0:6.3f} & {1:6.3f}".format(100*acceptance/total,100*acceptance/total)
            sel_temp = np.sum(T.array('weight')[sel_ev[k]])


            sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('METNoMuTrigger'))
            sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise')>=200)
            acc_met = np.sum(weight[sel_ev[k]])


        #     print("trigger + MET", accep_met[k]/acceptance, np.sum(weight[sel_ev[k]])/sel_temp, accep_met[k]/total,np.count_nonzero(sel_ev[k]))
            table["met"] += " & {0:6.3f} & {1:6.3f}".format(100*acc_met/acceptance, 100*acc_met/total)
            sel_temp = np.sum(weight[sel_ev[k]])



            sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('Flag2_all') == 1)
        #     print("MET filters", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
            table["met_filter"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
            sel_temp = np.sum(weight[sel_ev[k]])


            sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
        #     print("1 jet", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
            table["jet"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
            sel_temp = np.sum(weight[sel_ev[k]])



            sel_ev[k] = np.logical_and(sel_ev[k] , T.array("nDtRings")+T.array("nCscRings") < 10)
        #     print("cosmic shower veto", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
            table["cosmic_veto"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)

            sel_temp = np.sum(weight[sel_ev[k]])



            spike = np.logical_and( T.array('nDTRechitsSector')[:,0,0,7]>50,  T.array('nDTRechitsSector')[:,0,0,7]+T.array('nDTRechitsSector')[:,0,0,8]+T.array('nDTRechitsSector')[:,0,0,9]>120)
            spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,8]>25)
            spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,9]>10)
            sel_ev[k] = np.logical_and(sel_ev[k] , np.logical_not(spike))
        #     print("spike", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
            table["spike"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)

            sel_temp = np.sum(weight[sel_ev[k]])


            ########### SELECTION: CLUSTERS ############

            if category == 0:
                sel_rechitcluster = T.array('cscRechitCluster' + cluster_index + 'Size')>=50
                sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_gLLP_csc'))
        #         print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["2_cluster"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


                sel_rechitcluster = np.logical_and( sel_rechitcluster, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 100, T.array('cscRechitClusterMuonVetoGlobal'))))
        #         print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp) 
                table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


                sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') < 30)
        #         print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["jet_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
                


                sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'Me11Ratio') < 1)
        #         print("me11 ratio", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["me11_ratio"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
                m = create_TH1D(np.ones(len(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])), 'hm1', binning=[1,0,1000], weights = weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])
                
        
                sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') < 12.5, T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') > -5.0))
        #         print("time cut", np.sum([np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["time"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


                sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'TimeSpreadWeightedAll') < 20)
        #         print("time spread cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["time_spread"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])



                sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1.2)
        #         print("dphi cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["dphi"] +=  " & {0:6.2f} & {1:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

                temp = np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)
                cond = deltaR(T.array('cscRechitClusterEta')[sel_rechitcluster][temp][:,0], T.array('cscRechitClusterPhi')[sel_rechitcluster][temp][:,0],
                                                             T.array('cscRechitClusterEta')[sel_rechitcluster][temp][:,1], T.array('cscRechitClusterPhi')[sel_rechitcluster][temp][:,1])<2
                sel_ev[k][temp] = cond
        #         print("deltaR cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)

                table["deltaR"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

                sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'Size')) >= 100)
        #         print("nrechits cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["nrechits"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

                table['station_ratio'] += "&  /  & /"
                table['dt_cosmic_veto'] += "&  /  & /"

            if category == 1:
                sel_rechitcluster = T.array('dtRechitCluster' + cluster_index + 'Size')>=0
                sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster' + cluster_index + '_match_gLLP'))

#                 print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)

                table["2_cluster"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


                sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
                sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
                sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
                sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
                                           T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
                nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
                +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)

                sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))    
        #         print("dt cosmic veto cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["dt_cosmic_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])




                sel_rechitcluster = np.logical_and( sel_rechitcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))
        #         print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


                sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitCluster' + cluster_index + 'JetVetoPt') < 50)
        #         print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["jet_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

                sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))
        #         print("MB1 ratio", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["me11_ratio"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

                cut = 5
                station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
            +(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)
                max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
                min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
                sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_or(station<4, min_station/max_station<0.4)) #remove if both clusters are 4 stations
        #         print("min/max station ratio", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["station_ratio"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])


                sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('dtRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1)
        #         print("dphi cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["dphi"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])

                sel_rechitcluster = np.logical_and( sel_rechitcluster, np.abs(T.array('dtRechitCluster' + cluster_index + 'Size')) >= 80)
        #         print("nrechits cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)
                table["nrechits"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp,\
                                                              100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()==2)])
        #         print(np.count_nonzero(sel_temp))

                table['time'] += " & / & / "
                table['time_spread'] += " & / & /"

                table['deltaR'] += " & / & /"

            if category == 2:
                sel_csc = T.array('cscRechitCluster' + cluster_index + 'Size')>=50
                sel_dt = T.array('dtRechitCluster' + cluster_index + 'Size')>=50
                sel_csc = np.logical_and(sel_csc, T.array('cscRechitCluster' + cluster_index + '_match_gLLP'))
                sel_dt = np.logical_and(sel_dt, T.array('dtRechitCluster' + cluster_index + '_match_gLLP'))


        #         print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

                table["2_cluster"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])




                sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
                sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
                sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
                sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
                                           T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
                nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
                +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)

                sel_dt = np.logical_and(sel_dt, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))    
        #         print("dt cosmic veto cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=2)])/sel_temp)

                table["dt_cosmic_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])





                sel_dt = np.logical_and( sel_dt, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))
                sel_csc = np.logical_and( sel_csc, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 100, T.array('cscRechitClusterMuonVetoGlobal'))))
        #         print("muon veto", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

                table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


                sel_dt = np.logical_and( sel_dt, T.array('dtRechitCluster' + cluster_index + 'JetVetoPt') < 50)
                sel_csc = np.logical_and(sel_csc, T.array('cscRechitClusterJetVetoPt') < 30)
        #         print("jet veto", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

                table["jet_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


                sel_csc = np.logical_and( sel_csc, T.array('cscRechitClusterMe11Ratio') < 1)
                sel_dt = np.logical_and(sel_dt, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))
        #         print("mb1 ratio", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

                table["me11_ratio"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


                cut = 5
                station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
            +(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)
                max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
                min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
                sel_dt = np.logical_and(sel_dt, np.logical_or(station<4, min_station/max_station<0.4)) #remove if both clusters are 4 stations
        #         print("station cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

                table["station_ratio"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])



                sel_csc = np.logical_and(sel_csc, np.logical_and(T.array('cscRechitClusterTimeWeighted') < 12.5, T.array('cscRechitClusterTimeWeighted') > -5.0))
        #         print("time cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
                table["time"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


                sel_csc = np.logical_and(sel_csc, T.array('cscRechitClusterTimeSpreadWeightedAll')<20)
        #         print("time spread", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)
                table["time_spread"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


                sel_dt = np.logical_and( sel_dt, np.abs(T.array('dtRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1)
                sel_csc = np.logical_and( sel_csc, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi')) < 1.2)
        #         print("dphi cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

                table["dphi"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])

                temp = np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))
                cond = deltaR(T.array('dtRechitClusterEta')[sel_dt][temp][:,0], T.array('dtRechitClusterPhi')[sel_dt][temp][:,0],
                                                             T.array('cscRechitClusterEta')[sel_csc][temp][:,0], T.array('cscRechitClusterPhi')[sel_csc][temp][:,0])<2.5

                sel_ev[k][temp] = cond

        #         print("deltaR cut", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

                table["deltaR"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  

                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])


                sel_dt = np.logical_and( sel_dt, np.abs(T.array('dtRechitCluster' + cluster_index + 'Size')) >= 80)
                sel_csc = np.logical_and( sel_csc, np.abs(T.array('cscRechitCluster' + cluster_index + 'Size')) >= 100)
        #         print("nrechits", np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp)

                table["nrechits"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/sel_temp,\
                                                                                 100*np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()>=1, sel_dt.sum()>=1))])/acc_met)  
                sel_temp = np.sum(weight[np.logical_and(sel_ev[k], np.logical_and(sel_csc.sum()==1, sel_dt.sum()==1))])
        #         print(np.count_nonzero(sel_temp))
#             print(category, sel_temp)

            #### print start of table
        if 'dd' in k:
            decay = '2 d quarks'
            name = 'dd'
        if 'BB' in k:
            decay = '2 b quarks'
            name = 'bb'
        if 'TauTau' in k:
            decay = '4$\\tau$ leptons'
            name = 'TauTau'
        if 'K0K0' in k:
            decay = '2 $K^{0}$'
            name = 'k0'
        if 'KPlusKMinus' in k:
            decay = '$K^{+}K^{-}$'
            name = 'kpkm'
        if 'Pi0' in k:
            decay = '2 $\\pi^{0}$'
            name = 'pi0'
        if 'PiPlusPiMinus' in k:
            decay = '$\\pi^{+}\pi^{-}$'
            name = 'pippim'
        if 'EE' in k:
            decay = '2 electrons'
            name = 'ee'
        if 'Gamma' in k:
            decay = '2 photons'
            name = 'gg'
    
        print("\\begin{table}[!h]")
        print("\\begin{center}")
       
        ct_m = str(int(float(ct)/1000))
        if int(ct)<1000:ct_m = str(round(float(ct)/1000,1))
        
        print("\\caption{Signal Efficiency(\\%) of each cut for twin higgs model, LLP mass "+mass+" GeV, lifetime of "+ct+" mm and decaying to "+decay+". \
(Cumulative Efficiency for all cuts applied after Trigger and MET cut are calculated with \
respect to events in acceptance and pass the MET and trigger cut and cut efficiency is calculated \
with respect to the previous cut. Acceptance is defined as when at least one LLP decays inside CSC.)}")
        print("\\label{tab:selections_sig_"+mass+"_"+ct+"_"+name+"}")
        
        print("\\begin{tabular}{c|c c | c c| cc}")
        print("\\hline")
        print(" & \\multicolumn{2}{c}{CSC-CSC} & \\multicolumn{2}{c}{DT-DT} & \\multicolumn{2}{c}{DT-CSC} \\\\")
        print("\\hline")
        print("Selection & cut eff & overall eff & cut eff & overall eff & cut eff & overall eff \\\\")
        print("\\hline")
            
        for k,v in table.items():
            #print table
            print(v+r" \\")


         #### print end of table
        print("\\hline")
        print("\\end{tabular}")
        print("\\end{center}")
        print("\\end{table}")
        print("\n")

\begin{table}[!h]
\begin{center}
\caption{Signal Efficiency(\%) of each cut for twin higgs model, LLP mass 7 GeV, lifetime of 1000 mm and decaying to 2 d quarks. (Cumulative Efficiency for all cuts applied after Trigger and MET cut are calculated with respect to events in acceptance and pass the MET and trigger cut and cut efficiency is calculated with respect to the previous cut. Acceptance is defined as when at least one LLP decays inside CSC.)}
\label{tab:selections_sig_7_1000_dd}
\begin{tabular}{c|c c | c c| cc}
\hline
 & \multicolumn{2}{c}{CSC-CSC} & \multicolumn{2}{c}{DT-DT} & \multicolumn{2}{c}{DT-CSC} \\
\hline
Selection & cut eff & overall eff & cut eff & overall eff & cut eff & overall eff \\
\hline
acceptance &  1.477 &  1.477 &  4.509 &  4.509 &  4.912 &  4.912 \\
Trigger and MET  &  0.498 &  0.007 &  0.653 &  0.029 &  0.468 &  0.023 \\
MET filters  &  98.53 &  98.53 &  96.29 &  96.29 &  98.14 &  98.14 \\
$N_{jet} \geq 1$  &  96.34 &  94.93 &  97.40 &  93.79 &  96.43 &  94.

Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hm1 (Potential memory leak).
Warning in <TROOT::Append>: 